In [15]:
import socket
import serial
import time
import numpy as np



# UWB

In [16]:
portU = "/dev/ttyACM2" #PC에 연결된 포트명
baudU = 115200  # 시리얼 보드레이트(통신속도)
try:
    serU = serial.Serial('/dev/ttyACM2', 115200, timeout=1)
    print("Connect _ UWB")
except:
    print("Not Connect _ UWB _")



Not Connect _ UWB _


In [17]:
while True:         
    print(serU.readlines())

SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

# MD

In [18]:
RMID = 183          # RMID
TMID = 184          # TMID
ID = 1              # ID
PID_MAINBC = 10     # PID (MAINBC)
PID_CMD = 207       # PID (207 = RPM)
DATA_NUM = 7        # PID 207 DATA_NUM
WHEEL_L_ENABLE = 1  # L모터 Enable
WHEEL_R_ENABLE = 1  # R모터 Enable
DUMMY = 0           # CRC 전 DUMMY

portM = "/dev/ttyMD400" #PC에 연결된 포트명
baudM = 57600  # 시리얼 보드레이트(통신속도)
try:
    serM = serial.Serial(portM, baudM, timeout=1)
    print("Connect _ MD400T")
except:
    print("Not Connect _ MD400T _")
    

def speed_msg(L_SPD, R_SPD): # SPEED MSG
    MOTOR_L_SPD = [L_SPD & 255,(L_SPD >> 8)&255] # SPEED LOW
    MOTOR_R_SPD = [R_SPD & 255,(R_SPD >> 8)&255] # SPEED HI
    send_msg = [RMID, TMID, ID, PID_CMD, DATA_NUM, WHEEL_L_ENABLE, MOTOR_L_SPD[0],MOTOR_L_SPD[1],WHEEL_R_ENABLE,MOTOR_R_SPD[0],MOTOR_R_SPD[1],DUMMY]
    CHC = (~sum(send_msg)+1)&255 # CHECKSUM
    send_msg.append(CHC) # CHC 추가
    serM.write(send_msg) # MD400T 송신
    return

def stop_msg(): # STOP MSG
    return speed_msg(0,0)

def monitoring_bc(on): # MAIN DATA monitoring ON(1) / OFF(0)
    if on == 1:
        send_msg = [RMID, TMID, ID, 10, 1, 5, 128]
        return serM.write(send_msg)
    elif on == 0:
        send_msg = [RMID, TMID, ID, 10, 1, 6, 127]
        return serM.write(send_msg)
    
def data_monitor(n): # MAIN DATA Monitoring 값 추출
    return bytearray(serM.read(23))[n]

def vel_speed_msg(left, right):
    rad = 0.1375            # 바퀴 반지름
    rpm = (2*3.14*rad)      # 한바퀴 돌아갈때 거리 약 0.83m
    spdL= left*60
    spdR= right*60    
    vel_spd_msg_L = int((spdL/rpm)*15)
    vel_spd_msg_R = int((spdR/rpm)*15)
    speed_msg(vel_spd_msg_L,-vel_spd_msg_R)
    return
    
def tq_off_msg():
    send_msg = [RMID, TMID, ID, 5, 1, 1, 137]
    serM.write(send_msg) # MD400T 송신
    return


Connect _ MD400T


In [19]:
#vel_speed_msg(0.5,0.5)
stop_msg()

# UDP (CV)

In [39]:
import socket

localIP     = "127.0.0.1"
localPort   = 9800
bufferSize  = 1024

msgFromServer       = "Hello UDP Client"
bytesToSend         = str.encode(msgFromServer)

# 데이터그램 소켓을 생성
UDPServerSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)

# 주소와 IP로 Bind
UDPServerSocket.bind((localIP, localPort))

print("UDP server up and listening")

#UWB_DIST = float(serU.readline().decode().replace('\n',''))

vel_speed_msg(0.2,0.2)
# 들어오는 데이터그램 Listen
while(True):
    UWB_DIST = 0.5
    bytesAddressPair = UDPServerSocket.recvfrom(bufferSize)
    message = bytesAddressPair[0]
    address = bytesAddressPair[1]
    
    direction = int(message.decode())
    # Sending a reply to client
    UDPServerSocket.sendto(bytesToSend, address)
    print(direction)
    
    if serU.readable(): #값이 들어왔는지 확인          
        if UWB_DIST <= 2.0: # UWB거리가 2m 이내일때
            print(direction)
            if direction > 260 and direction < 380: # 센터일때 멈춤
                stop_msg()
                print(' STOP, UNDER 2.0M ', 'CENTER ',UWB_DIST, direction)
            elif direction <= 310:
                vel_speed_msg(-0.10, 0.10)
                print(' STOP, TURN LEFT ', 'DIST = ', UWB_DIST, direction)
            elif direction >= 330:
                vel_speed_msg(0.10, -0.10)
                print(' STOP, TURN RIGHT ', 'DIST = ', UWB_DIST, direction)
        elif UWB_DIST > 2.0:
            if direction > 310 or direction < 330: # 센터일때 멈춤
                vel_speed_msg(0.4, 0.4)
                print(' GO, OVER 2.0M ', 'DIST = ',UWB_DIST, direction)
            elif direction < 310:
                vel_speed_msg(0.4, 0.6)
                print(' GO, TURN RIGHT ', 'DIST = ', UWB_DIST, direction)
            elif direction > 330:
                vel_speed_msg(0.6, 0.4)
                print(' GO, TURN RIGHT ', 'DIST = ', UWB_DIST, direction)

    

UDP server up and listening
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
622
622
 STOP, TURN RIGHT  DIST =  0.5 622
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
592
592
 STOP, TURN RIGHT  DIST =  0.5 592
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
200
200
 STOP, TURN LEFT  DIST =  0.5 200
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
39
39
 STOP, TURN LEFT  DIST =  0.5 39
27
27
 STOP, TURN LEFT  DIST =  0.5 27
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP, TURN LEFT  DIST =  0.5 0
0
0
 STOP,

KeyboardInterrupt: 

In [ ]:
vel_speed_msg(-)

In [36]:
stop_msg()

In [38]:
tq_off_msg()